## Simple cell lines cryobank database 

In [22]:
import sqlite3 as sql
import pandas as pd

In [23]:
# load previously created table with cell lines description via pandas
cell_lines_descr_df = pd.read_csv('cell_lines_description.csv')
cell_lines_descr_df.head()

,cell_line_id,cell_line_name,cell_type,origin,sourse_of_line
0,1,A549,"epithelial, lung adenocarcinoma",human,ATCC CCL-185
1,2,SK-Hep1,"endothelial, liver adenocarcinoma",human,ATCC HTB-52
2,3,SK-UT-1B,"mesenchymal, uterus leiomyosarcoma",human,ATCC HTB-115
3,4,bmMSCs,"mesenchymal, bone marrow",mouse,primary_line
4,5,adMSCs,"mesenchymal, adipose tissue",human,primary_line


In [24]:
# create and connect to a database
connection = sql.connect('cell_lines_cryobank.db') 

In [25]:
# create table 'cell_lines_description' 
connection.execute('''CREATE TABLE IF NOT EXISTS cell_lines_description
(cell_line_id INTEGER PRIMARY KEY,
cell_line_name TEXT,
cell_type TEXT,
origin TEXT,
sourse_of_line TEXT )''')
connection.commit()

In [26]:
# load data into the table via pd dataframe
cell_lines_descr_df.to_sql('cell_lines_description', 
                           connection, 
                           if_exists='append', 
                           index = False)
connection.commit()

In [31]:
# create and fill in the table 'cryo_tanks_description' manually, not via pd dataframe 
connection.execute('''CREATE TABLE IF NOT EXISTS cryo_tanks_description
(cryo_tank_id INTEGER PRIMARY KEY,
location TEXT,
temperature INTEGER,
access TEXT)''')
connection.commit()

# add data to the table 'cryo_tanks_description'
cryo_tanks = [('Tank 1', -160, 'free'),
              ('Tank 2', -80, 'free'),
              ('Tank 3', -160, 'limited')]
query = "INSERT INTO cryo_tanks_description (location, temperature, access) " \
        "VALUES (?, ?, ?)"
connection.executemany(query, cryo_tanks)
connection.commit()

In [50]:
# look at the produced 'cryo_tanks_description' table
query = """ SELECT *
            FROM cryo_tanks_description """

df = pd.read_sql_query(query, connection)
df.head()

,cryo_tank_id,location,temperature,access
0,1,Tank 1,-160,free
1,2,Tank 2,-80,free
2,3,Tank 3,-160,limited


In [32]:
# create manually one more table 'culture_medium_conditions'
connection.execute('''CREATE TABLE IF NOT EXISTS culture_medium_conditions
(culture_medium_id INTEGER PRIMARY KEY,
basic_medium TEXT,
serum TEXT,
antibiotics TEXT,
additives TEXT)''')
connection.commit()

# add data to the table 'cryo_tanks_description'
culture_mediums = [('DMEM', 'FBS, 10%', 'Gentamicin, 1%', None),
                   ('DMEM/F12', 'FBS, 10%', 'Penicillin/Streptomicin, 1%', 'Glulamine, 3%')]
query = "INSERT INTO culture_medium_conditions (basic_medium, serum, antibiotics, additives) " \
        "VALUES (?, ?, ?, ?)"
connection.executemany(query, culture_mediums)
connection.commit()

In [51]:
# look at the produced 'culture_medium_conditions' table
query = """ SELECT *
            FROM culture_medium_conditions """

df = pd.read_sql_query(query, connection)
df.head()

,culture_medium_id,basic_medium,serum,antibiotics,additives
0,1,DMEM,"FBS, 10%","Gentamicin, 1%",None
1,2,DMEM/F12,"FBS, 10%","Penicillin/Streptomicin, 1%","Glulamine, 3%"


In [34]:
# CASCADE

# create summary table 'cell_lines_cryobank'
connection.execute('''CREATE TABLE IF NOT EXISTS cell_lines_cryobank
(cell_lines_cryobank INTEGER PRIMARY KEY,
cell_line_id INTEGER,
cryo_tank_id INTEGER,
culture_medium_id INTEGER,
FOREIGN KEY (cell_line_id) REFERENCES cell_lines_description (cell_line_id) ON DELETE CASCADE,
FOREIGN KEY (cryo_tank_id) REFERENCES cryo_tanks_description (cryo_tank_id) ON DELETE CASCADE,
FOREIGN KEY (culture_medium_id) REFERENCES culture_medium_conditions (culture_medium_id) ON DELETE CASCADE )''')
connection.commit()

# add data to the table 'cell_lines_cryobank'
cell_lines_cryobank_data = [(1, 1, 1),
                       (2, 1, 1),
                       (3, 2, 1),
                       (4, 3, 2),
                       (5, 3, 2)]
query = "INSERT INTO cell_lines_cryobank (cell_line_id, cryo_tank_id, culture_medium_id) " \
        "VALUES (?, ?, ?)"
connection.executemany(query, cell_lines_cryobank_data)
connection.commit()

In [44]:
# JOIN and SELECT

# SELECT all the data to check the database was constructed correctly
query = ''' SELECT * 
            FROM cell_lines_cryobank
            JOIN cell_lines_description USING (cell_line_id)
            JOIN cryo_tanks_description USING (cryo_tank_id)
            JOIN culture_medium_conditions USING (culture_medium_id)'''

df = pd.read_sql_query(query, connection)
df.head()

,cell_lines_cryobank,cell_line_id,cryo_tank_id,culture_medium_id,cell_line_name,cell_type,origin,sourse_of_line,location,temperature,access,basic_medium,serum,antibiotics,additives
0,1,1,1,1,A549,"epithelial, lung adenocarcinoma",human,ATCC CCL-185,Tank 1,-160,free,DMEM,"FBS, 10%","Gentamicin, 1%",None
1,2,2,1,1,SK-Hep1,"endothelial, liver adenocarcinoma",human,ATCC HTB-52,Tank 1,-160,free,DMEM,"FBS, 10%","Gentamicin, 1%",None
2,3,3,2,1,SK-UT-1B,"mesenchymal, uterus leiomyosarcoma",human,ATCC HTB-115,Tank 2,-80,free,DMEM,"FBS, 10%","Gentamicin, 1%",None
3,4,4,3,2,bmMSCs,"mesenchymal, bone marrow",mouse,primary_line,Tank 3,-160,limited,DMEM/F12,"FBS, 10%","Penicillin/Streptomicin, 1%","Glulamine, 3%"
4,5,5,3,2,adMSCs,"mesenchymal, adipose tissue",human,primary_line,Tank 3,-160,limited,DMEM/F12,"FBS, 10%","Penicillin/Streptomicin, 1%","Glulamine, 3%"


In [49]:
# JOIN and SELECT WHERE

# SELECT specific colunms only for cell lines with free access
query = """ SELECT cell_line_name, 
                   origin, 
                   location, 
                   basic_medium, 
                   serum, 
                   antibiotics, 
                   additives, 
                   access
            FROM cell_lines_cryobank 
            JOIN cell_lines_description USING (cell_line_id)
            JOIN cryo_tanks_description USING (cryo_tank_id)
            JOIN culture_medium_conditions USING (culture_medium_id)
            WHERE access = 'free' """

df = pd.read_sql_query(query, connection)
df.head()

,cell_line_name,origin,location,basic_medium,serum,antibiotics,additives,access
0,A549,human,Tank 1,DMEM,"FBS, 10%","Gentamicin, 1%",None,free
1,SK-Hep1,human,Tank 1,DMEM,"FBS, 10%","Gentamicin, 1%",None,free
2,SK-UT-1B,human,Tank 2,DMEM,"FBS, 10%","Gentamicin, 1%",None,free


In [53]:
# DELETE and UPDATE some records

# DELETE cell line SK-Hep1 
connection.execute("PRAGMA foreign_keys = on")
query = """DELETE FROM cell_lines_description 
                  WHERE cell_line_name = 'SK-Hep1' """
connection.execute(query)
connection.commit()

# UPDATE serum requirements for the first culture medium
connection.execute("PRAGMA foreign_keys = on")
query = """UPDATE culture_medium_conditions 
           SET serum = 'FBS, 5%' 
           WHERE culture_medium_id = 1 """
connection.execute(query)
connection.commit()

# update SELECT specific colunms only for cell lines with free access
query = """ SELECT cell_line_name, 
                   origin, 
                   location, 
                   basic_medium, 
                   serum, 
                   antibiotics, 
                   additives, 
                   access
            FROM cell_lines_cryobank 
            JOIN cell_lines_description USING (cell_line_id)
            JOIN cryo_tanks_description USING (cryo_tank_id)
            JOIN culture_medium_conditions USING (culture_medium_id)
            WHERE access = 'free' """

df = pd.read_sql_query(query, connection)
df.head()

,cell_line_name,origin,location,basic_medium,serum,antibiotics,additives,access
0,A549,human,Tank 1,DMEM,"FBS, 5%","Gentamicin, 1%",None,free
1,SK-UT-1B,human,Tank 2,DMEM,"FBS, 5%","Gentamicin, 1%",None,free


In [54]:
connection.close()